In [2]:
#Import the required packages
#Import package pandas for data analysis
import pandas as pd

# Import package numpy for numeric computing
import numpy as np

# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt

#For showing plots directly in the notebook run the command below
%matplotlib inline

# For saving multiple plots into a single pdf file
from matplotlib.backends.backend_pdf import PdfPages 

#to load the MAPPluto data
import geopandas as gpd

from shapely.geometry import Point

from haversine import haversine, Unit

In [3]:
df = pd.read_csv('MTA_Subway_Entrances_and_Exits__2024_20250524.csv', keep_default_na=True, delimiter=',', skipinitialspace=True)
#How many rows should be displayed in full
pd.set_option('display.max_rows', 100)
# Show data frame first few rows
df.head()

,Division,Line,Borough,Stop Name,Complex ID,Constituent Station Name,Station ID,GTFS Stop ID,Daytime Routes,Entrance Type,Entry Allowed,Exit Allowed,Entrance Latitude,Entrance Longitude,entrance_georeference
0,BMT,4th Av,B,Atlantic Av-Barclays Ctr,617,Atlantic Av-Barclays Ctr,27,R31,2 3 4 5 B D N Q R,Stair,YES,YES,40.683905,-73.978879,POINT (-73.978879 40.683905)
1,BMT,4th Av,B,Atlantic Av-Barclays Ctr,617,Atlantic Av-Barclays Ctr,27,R31,2 3 4 5 B D N Q R,Elevator,YES,YES,40.683805,-73.978487,POINT (-73.978487 40.683805)
2,BMT,4th Av,B,Atlantic Av-Barclays Ctr,617,Atlantic Av-Barclays Ctr,27,R31,2 3 4 5 B D N Q R,Stair,YES,YES,40.683928,-73.978412,POINT (-73.978412 40.683928)
3,BMT,4th Av,B,Union St,28,Union St,28,R32,R,Stair,YES,YES,40.677154,-73.983430,POINT (-73.9834296 40.6771544)
4,BMT,4th Av,B,Union St,28,Union St,28,R32,R,Stair,YES,YES,40.677296,-73.983336,POINT (-73.9833364 40.6772958)


For the Mobility Score we need the subway access. Therefore the following fetaures are relevant:
- Stop Name
- Entry Allowed
- Emtrance Type (for accessibility)
- Daytime Routes
- Entrance Latitude
- Entrance Longitude

In [4]:
#filter for Manhatten first

df['Borough'].unique()


array(['B', 'Q', 'M', 'Bx', 'SI'], dtype=object)

In [5]:
df = df[df['Borough']== 'M']

In [6]:
df.shape

(868, 15)

In [7]:
df = df.drop('Borough', axis=1)

In [8]:
#only select relevant features
df = df[['Stop Name','Entry Allowed', 'Entrance Type', 'Daytime Routes','Exit Allowed', 'Entrance Latitude', 'Entrance Longitude']]

In [9]:
#change datatype
df.dtypes

Stop Name              object
Entry Allowed          object
Entrance Type          object
Daytime Routes         object
Exit Allowed           object
Entrance Latitude     float64
Entrance Longitude    float64
dtype: object

In [10]:
df.head()

,Stop Name,Entry Allowed,Entrance Type,Daytime Routes,Exit Allowed,Entrance Latitude,Entrance Longitude
52,Roosevelt Island,YES,Station House,F,YES,40.759019,-73.953458
53,Lexington Av/63 St,YES,Easement - Street,F Q,YES,40.764968,-73.966679
54,Lexington Av/63 St,YES,Stair,F Q,YES,40.764738,-73.966553
55,Lexington Av/63 St,YES,Easement - Street,F Q,YES,40.764896,-73.966426
56,Lexington Av/63 St,YES,Stair,F Q,YES,40.764101,-73.965041


In [11]:
#check for duplicates
df[df.duplicated()]

,Stop Name,Entry Allowed,Entrance Type,Daytime Routes,Exit Allowed,Entrance Latitude,Entrance Longitude


In [12]:
df.isnull().sum()

Stop Name             0
Entry Allowed         0
Entrance Type         0
Daytime Routes        0
Exit Allowed          0
Entrance Latitude     0
Entrance Longitude    0
dtype: int64

In [13]:
df.shape

(868, 7)

In [14]:
#Get the Target Points from MapPluto

pluto = gpd.read_file('../location_data/MapPLUTO.shp')

In [15]:
pluto.head()

,Borough,Block,Lot,CD,BCT2020,BCTCB2020,CT2010,CB2010,SchoolDist,Council,...,FIRM07_FLA,PFIRM15_FL,Version,DCPEdited,Latitude,Longitude,Notes,Shape_Leng,Shape_Area,geometry
0,MN,1,10,101,1000500,10005000003,5,1000,02,1,...,1,1,25v1.1,None,40.688774,-74.018704,None,0.0,7.414502e+06,"POLYGON ((980898.728 191409.779, 980881.798 19..."
1,MN,1,101,101,1000100,10001001001,1,1001,02,1,...,None,1,25v1.1,None,40.689920,-74.045337,None,0.0,5.018973e+05,"MULTIPOLYGON (((972428.829 190679.175, 972443...."
2,MN,1,201,101,1000100,10001001000,1,1000,02,1,...,None,1,25v1.1,None,40.698188,-74.041329,None,0.0,1.148539e+06,"POLYGON ((973648.066 193711.894, 973525.342 19..."
3,MN,2,1,101,1000900,10009001022,9,1025,02,1,...,1,1,25v1.1,t,40.700369,-74.012911,None,0.0,1.008250e+05,"POLYGON ((980609.55 194220.422, 980608.726 194..."
4,MN,2,2,101,1000900,10009001022,9,1025,02,1,...,1,1,25v1.1,None,40.700550,-74.011588,None,0.0,8.724419e+04,"POLYGON ((980854.138 194531.437, 980823.199 19..."


In [16]:
pluto.shape

(856734, 95)

In [54]:
pluto['Borough'].unique()

array(['MN', 'BX', 'BK', 'QN', 'SI'], dtype=object)

In [17]:
pluto= pluto[pluto['Borough']== 'MN']

In [56]:
print(pluto.columns)

Index(['Borough', 'Block', 'Lot', 'CD', 'BCT2020', 'BCTCB2020', 'CT2010',
       'CB2010', 'SchoolDist', 'Council', 'ZipCode', 'FireComp', 'PolicePrct',
       'HealthCent', 'HealthArea', 'Sanitboro', 'SanitDistr', 'SanitSub',
       'Address', 'ZoneDist1', 'ZoneDist2', 'ZoneDist3', 'ZoneDist4',
       'Overlay1', 'Overlay2', 'SPDist1', 'SPDist2', 'SPDist3', 'LtdHeight',
       'SplitZone', 'BldgClass', 'LandUse', 'Easements', 'OwnerType',
       'OwnerName', 'LotArea', 'BldgArea', 'ComArea', 'ResArea', 'OfficeArea',
       'RetailArea', 'GarageArea', 'StrgeArea', 'FactryArea', 'OtherArea',
       'AreaSource', 'NumBldgs', 'NumFloors', 'UnitsRes', 'UnitsTotal',
       'LotFront', 'LotDepth', 'BldgFront', 'BldgDepth', 'Ext', 'ProxCode',
       'IrrLotCode', 'LotType', 'BsmtCode', 'AssessLand', 'AssessTot',
       'ExemptTot', 'YearBuilt', 'YearAlter1', 'YearAlter2', 'HistDist',
       'Landmark', 'BuiltFAR', 'ResidFAR', 'CommFAR', 'FacilFAR', 'BoroCode',
       'BBL', 'CondoNo', 'Tract2

In [18]:
# Make sure geometries are valid
pluto = pluto[pluto.is_valid]

# Group by Census Tract (if multiple parcels per tract) and compute unified geometry
tracts = pluto.dissolve(by="CT2010")

# Compute centroids
tracts["centroid"] = tracts.geometry.centroid

# Extract lat/lon from centroids for use in scoring
tracts["latitude"] = tracts["centroid"].y
tracts["longitude"] = tracts["centroid"].x

In [19]:
#Store the Target Points (centroids in a dataframe)
centroids_df = gpd.GeoDataFrame(
    tracts,
    geometry=tracts["centroid"],
    crs=tracts.crs
)

In [30]:
#Computing Nearby Access

def count_entrances_near(lat, lon, entrances, radius_m=500):
    count = 0
    unique_lines = set()
    ada_count = 0
    for _, row in entrances.iterrows():
        dist = haversine(
            (lat, lon), 
            (row["Entrance Latitude"], row["Entrance Longitude"]),
            unit="m"
        )
        if dist <= radius_m:
            count += 1
            unique_lines.update(str(row["Daytime Routes"]).split())
            if "elevator" in str(row["Entrance Type"]).lower():
                ada_count += 1
    return count, len(unique_lines), ada_count



In [75]:
print(df.columns.tolist())


['Stop Name', 'Entry Allowed', 'Entrance Type', 'Daytime Routes', 'Exit Allowed', 'Entrance Latitude', 'Entrance Longitude']


In [31]:
#check nearby accesses in the centroids
centroids_df = centroids_df.to_crs(epsg=4326)

# Assuming centroids_df has columns: ["geometry", "CensusTract"]
centroids_df["latitude"] = centroids_df.geometry.y
centroids_df["longitude"] = centroids_df.geometry.x

# Storage for scores
results = []

for idx, row in centroids_df.iterrows():
    lat, lon = row["latitude"], row["longitude"]
    count, unique_lines, ada_count = count_entrances_near(lat, lon, df)
    results.append((count, unique_lines, ada_count))

centroids_df[["entrance_count", "line_count", "accessible_count"]] = pd.DataFrame(results, index=centroids_df.index)


In [32]:
#Get Subway Score
# Example weights (customize as needed)
w1, w2, w3 = 0.5, 0.3, 0.2

centroids_df["raw_score"] = (
    w1 * centroids_df["entrance_count"] +
    w2 * centroids_df["line_count"] +
    w3 * centroids_df["accessible_count"]
)


In [33]:
# Normalize to scale 1–10
min_score = centroids_df["raw_score"].min()
max_score = centroids_df["raw_score"].max()

centroids_df["subway_score"] = 1 + 9 * (centroids_df["raw_score"] - min_score) / (max_score - min_score)
#centroids_df["subway_score"] = centroids_df["subway_score"].round(2)  # Optional


In [34]:
centroids_df.head()

,geometry,Borough,Block,Lot,CD,BCT2020,BCTCB2020,CB2010,SchoolDist,Council,...,Shape_Leng,Shape_Area,centroid,latitude,longitude,entrance_count,line_count,accessible_count,raw_score,subway_score
CT2010,,,,,,,,,,,,,,,,,,,,,
1,POINT (-74.04224 40.69604),MN,1,101,101,1000100,10001001001,1001,02,1,...,0.0,5.018973e+05,POINT (972538.031 192864.582),40.696038,-74.042236,0,0,0,0.0,1.000000
10.01,POINT (-73.97503 40.71814),MN,316,200,103,1001001,10010011014,1000,01,1,...,0.0,1.469959e+06,POINT (991172.651 200915.69),40.718141,-73.975027,0,0,0,0.0,1.000000
10.02,POINT (-73.97774 40.71748),MN,323,1,103,1001002,10010022001,2002,01,2,...,0.0,1.210255e+06,POINT (990421.877 200676.354),40.717485,-73.977736,0,0,0,0.0,1.000000
100,POINT (-73.97118 40.75807),MN,1304,20,106,1010000,10100002006,2006,02,4,...,0.0,2.510072e+04,POINT (992233.273 215461.632),40.758065,-73.971184,24,3,0,12.9,3.379098
101,POINT (-73.99158 40.74972),MN,780,2,105,1010100,10101001004,1005,02,3,...,0.0,2.789032e+03,POINT (986584.318 212419.961),40.749720,-73.991575,50,14,2,29.6,6.459016


In [ ]:
#store cleaned data set as csv
#centroids_df.to_file("manhatten_subway_scores.geojson", driver="GeoJSON")
#centroids_df.to_csv("manhatten_subway_scores.csv", index=False)